#0. 환경

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when, count, regexp_extract, regexp_replace, row_number
from pyspark.ml.functions import vector_to_array
from pyspark.ml.feature import RegexTokenizer, CountVectorizer, VectorAssembler, MinMaxScaler, StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.window import Window
import numpy as np
import pandas as pd
from xgboost import XGBClassifier

In [0]:
spark = SparkSession.builder \
    .appName("XGBoost") \
    .getOrCreate()

# 1. 데이터로드

### 데이터 로드

In [0]:
catalog = "1dt_team8_databricks"
schema = "`final`"
path = f"{catalog}.{schema}"

try:
    train = spark.read.table(f"{path}.train_df")
    validation = spark.read.table(f"{path}.validation_df")
    test = spark.read.table(f"{path}.test_df")
except Exception as e:
    print(f"Error loading data from Unity Catalog Volume: {e}")
# display(train)
# display(validation)
# display(test)

train = train.withColumn("label", when(train["rating"] >= 4, 1).otherwise(0))
validation = validation.withColumn("label", when(validation["rating"] >= 4, 1).otherwise(0))
test = test.withColumn("label", when(test["rating"] >= 4, 1).otherwise(0))

### IMDB 로드

In [0]:
catalog = "1dt_team8_databricks"
schema = "`imdb`"
imdb_path = f"{catalog}.{schema}"

try:
    imdb_ratings = spark.read.table(f"{imdb_path}.title_ratings")
    print("Data loaded successfully from Unity Catalog Volume.")
except Exception as e:
    print(f"Error loading data from Unity Catalog Volume: {e}")
    print(f"Please ensure CSV files (imdb_ratings.csv) exist in {imdb_path}")

# 2. 데이터분리

## 파이프라인

In [0]:
tokenizer = RegexTokenizer(inputCol="genres", outputCol="genres_tokens", pattern="\\|")
vectorizer = CountVectorizer(inputCol="genres_tokens", outputCol="genres_vec")
# assembler_numvotes = VectorAssembler(inputCols=["numVotes"], outputCol="numVotes_vec")
# scaler = MinMaxScaler(inputCol="numVotes_vec", outputCol="numVotes_scaled")
user_indexer = StringIndexer(inputCol="userId", outputCol="userIndex")
user_indexer_model = user_indexer.fit(train)
movie_indexer = StringIndexer(inputCol="movieId", outputCol="movieIndex")

assembler_all = VectorAssembler(
    # inputCols=["genres_vec", "averageRating", "numVotes_scaled", "userIndex", "movieIndex", "year"],
    inputCols=["genres_vec", "userIndex", "movieIndex"],
    outputCol="features"
)

pipeline = Pipeline(stages=[
    # tokenizer, vectorizer, assembler_numvotes, scaler,
    # user_indexer, movie_indexer, assembler_all
    tokenizer, vectorizer, 
    user_indexer, movie_indexer, assembler_all
])

pipeline_model = pipeline.fit(train)
train_transformed = pipeline_model.transform(train)
validation_transformed = pipeline_model.transform(validation)
test_transformed = pipeline_model.transform(test)

In [0]:
total_count = train_transformed.count()
chunk_size = total_count // 3
window_spec = Window.orderBy("userId")  # 유일한 컬럼 기준 정렬

df_with_rownum = train_transformed.withColumn("row_num", row_number().over(window_spec))

dfs = []
for i in range(3):
    start = i * chunk_size + 1
    end = (i + 1) * chunk_size if i < 2 else total_count

    chunk_df = df_with_rownum.filter((col("row_num") >= start) & (col("row_num") <= end)).drop("row_num")
    chunk_df = chunk_df.withColumn("features_array", vector_to_array("features"))

    # ✅ 필요한 컬럼들 선택
    chunk_pd = chunk_df.select("userIndex", "movieIndex", "features_array", "label").toPandas()
    dfs.append(chunk_pd)

train_pd = pd.concat(dfs, ignore_index=True)

In [0]:
val_total_count = validation_transformed.count()
val_chunk_size = val_total_count // 3
val_window_spec = Window.orderBy("userId")  # 유일한 컬럼 기준 정렬

val_with_rownum = validation_transformed.withColumn("row_num", row_number().over(val_window_spec))

val_dfs = []
for i in range(3):
    start = i * val_chunk_size + 1
    end = (i + 1) * val_chunk_size if i < 2 else val_total_count

    chunk_df = val_with_rownum.filter((col("row_num") >= start) & (col("row_num") <= end)).drop("row_num")
    chunk_df = chunk_df.withColumn("features_array", vector_to_array("features"))

    # ✅ 필요한 컬럼들 선택
    chunk_pd = chunk_df.select("userIndex", "movieIndex", "features_array", "label").toPandas()
    val_dfs.append(chunk_pd)

validation_pd = pd.concat(val_dfs, ignore_index=True)

In [0]:
# 1. 총 개수와 청크 크기 정의
test_total_count = test_transformed.count()
test_chunk_size = test_total_count // 3
test_window_spec = Window.orderBy("userId")  # 고유 컬럼 기준 정렬

# 2. row_num 생성
test_with_rownum = test_transformed.withColumn("row_num", row_number().over(test_window_spec))

# 3. 3개로 나눠서 Pandas 변환 (필수 컬럼 포함)
test_dfs = []
for i in range(3):
    start = i * test_chunk_size + 1
    end = (i + 1) * test_chunk_size if i < 2 else test_total_count

    chunk_df = test_with_rownum.filter((col("row_num") >= start) & (col("row_num") <= end)).drop("row_num")
    chunk_df = chunk_df.withColumn("features_array", vector_to_array("features"))
    
    # 필요한 컬럼 전부 선택
    chunk_pd = chunk_df.select("userIndex", "movieIndex", "features_array", "label").toPandas()
    test_dfs.append(chunk_pd)

# 4. 합치기
test_pd = pd.concat(test_dfs, ignore_index=True)

In [0]:
# X (features), y (labels) 분리
X_train = np.array(train_pd['features_array'].tolist())
y_train = train_pd['label'].astype(int).values

X_valid = np.array(validation_pd['features_array'].tolist())
y_valid = validation_pd['label'].astype(int).values

X_test = np.array(test_pd['features_array'].tolist())
y_test = test_pd['label'].astype(int).values

# 3. 모델 설계 및 평가

## 평가지표 정의

In [0]:
# Precision@K 계산 함수
def precision_at_k(df: pd.DataFrame, k: int = 5, threshold: float = 4.0) -> float:
    top_k = (
        df.sort_values(["userIndex", "y_pred_proba"], ascending=[True, False])
        .groupby("userIndex")
        .head(k)
    )
    top_k["hit"] = (top_k["y_true"] >= threshold).astype(int)
    user_precision = top_k.groupby("userIndex")["hit"].mean()
    return user_precision.mean()

In [0]:
def recall_at_k(df: pd.DataFrame, k: int = 5, threshold: float = 4.0) -> float:
    # 사용자별 실제 긍정(평점 >= threshold) 아이템 수
    user_positive_counts = df[df["y_true"] >= threshold].groupby("userIndex").size()

    # 사용자별 top-k 추천 추출
    top_k = (
        df.sort_values(["userIndex", "y_pred_proba"], ascending=[True, False])
        .groupby("userIndex")
        .head(k)
        .copy()
    )
    top_k["hit"] = (top_k["y_true"] >= threshold).astype(int)

    # 사용자별 top-k 추천에서 맞춘 아이템 수
    user_hits = top_k.groupby("userIndex")["hit"].sum()

    # 사용자별 recall 계산 (실제 긍정 아이템 수가 0인 경우 제외)
    recall_per_user = user_hits / user_positive_counts
    recall_per_user = recall_per_user.dropna()  # NaN 처리 (실제 긍정 아이템이 없는 사용자 제외)

    # 전체 사용자 평균 Recall@K 반환
    return recall_per_user.mean()

In [0]:
def ndcg_at_k(df: pd.DataFrame, k: int = 10) -> float:
    def dcg(relevances):
        return np.sum((2**relevances - 1) / np.log2(np.arange(2, relevances.size + 2)))

    ndcgs = []

    for user, group in df.groupby("userIndex"):
        # user별 y_pred_proba 기준 내림차순 정렬, top-k 추출
        top_k = group.sort_values("y_pred_proba", ascending=False).head(k)
        rel = top_k["y_true"].values
        
        # 이상적인 순서(내림차순 y_true 정렬)
        ideal_rel = np.sort(group["y_true"].values)[::-1][:k]
        
        actual_dcg = dcg(rel)
        ideal_dcg = dcg(ideal_rel)

        ndcg = actual_dcg / ideal_dcg if ideal_dcg > 0 else 0
        ndcgs.append(ndcg)

    return np.mean(ndcgs)

## XgbClassifier

### 모델 설계 및 실험

In [0]:
model = XGBClassifier(
    objective = 'binary:logistic',
    max_depth=6,
    n_estimators=100,
    learning_rate=0.1,
    early_stopping_rounds=50,
    seed=0,
    n_jobs=-1,
    use_label_encoder=False,
    eval_metric='logloss',
)

model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=True)

In [0]:
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:, 1]

test_pdf = test_pd.copy()
test_pdf["y_pred"] = y_pred
test_pdf["y_pred_proba"] = y_pred_proba
test_pdf["y_true"] = test_pdf["label"]

### 평가지표

In [0]:
p_at_10 = precision_at_k(test_pdf, k=10)
print(f"Precision@10: {p_at_10:.4f}")

r_at_10 = recall_at_k(test_pdf, k=10)
print(f"Recall@10: {r_at_10:.4f}")

ndcg_10 = ndcg_at_k(test_pdf, k=10)
print(f"NDCG@10: {ndcg_10:.4f}")

### 사용자별 추천 영화

In [0]:
K = 10

# 1. 사용자별 예측 상위 K개 영화 선택
window_spec = Window.partitionBy("userIndex").orderBy(col("y_pred_proba").desc())

ranked = test_pred.withColumn("rank", row_number().over(window_spec)) \
                  .filter(col("rank") <= K)

# 2. 추천 영화 정보 묶기 (movieIndex와 예측 확률)
ranked = ranked.withColumn("recommendation", struct(col("movieIndex").alias("movieId"), col("y_pred_proba").alias("pred_rating")))

# 3. 사용자별로 리스트로 그룹화
recommendations = ranked.groupBy("userIndex") \
    .agg(collect_list("recommendation").alias("recommendations")) \
    .orderBy("userIndex")

display(recommendations)

### Optuna

In [0]:
# 레이블 생성 (평점 4 이상은 1, 아니면 0)
# Optuna 목적 함수 정의
def objective(trial):
    # 하이퍼파라미터 설정
    param = {
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "use_label_encoder": False,
        "eval_metric": "logloss",
        "early_stopping_rounds" : 20
    }

    # XGBoost 모델 학습
    model = XGBClassifier(**param)
    model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)],  verbose=False)

    y_pred = model.predict_proba(X_test)[:,1]
    test_pdf = test_df.select("userIndex", "movieIndex", "rating").toPandas()
    test_pdf["y_pred_prob"] = y_pred
    test_pdf.rename(columns={"rating": "y_true"}, inplace=True)

    pred = spark.createDataFrame(test_pdf)

    window_spec = Window.partitionBy("userIndex").orderBy(col("y_pred_prob").desc())
    top_k = pred.withColumn("rank", row_number().over(window_spec)).filter(col("rank") <= K)

    # Precision@K 계산
    top_k = top_k.withColumn("y_true", (col("y_true") >= 4).cast("int"))
    precision_df = top_k.groupBy("userIndex").agg({"y_true": "avg"})
    avg_precision_at_k = precision_df.selectExpr("avg(`avg(y_true)`) as precision_at_k").collect()[0]["precision_at_k"]

    return avg_precision_at_k  # Optuna는 최소화를 수행하므로 음수 반환

In [0]:
study = optuna.create_study(direction="maximize")  # Precision@K를 최대화
study.optimize(objective, n_trials=30)  # 30번 튜닝

print("Best params:", study.best_params)
print("Best Precision@K:", study.best_value)

In [0]:
best_params = study.best_params

best_model = XGBClassifier(**best_params)
best_model.fit(X_train, y_train)

best_pred = best_model.predict_proba(X_test)[:,1]

# 1. 디렉토리 생성
os.makedirs("/dbfs/FileStore/models", exist_ok=True)

joblib.dump(best_model, "/dbfs/FileStore/models/best_xgb_clf_model.pkl")

# loaded_model = joblib.load("best_xgb_model.pkl")
# y_pred = loaded_model.predict(X_test)

### Optuna + mlflow

In [0]:
# 레이블 생성 (평점 4 이상은 1, 아니면 0)
# Optuna 목적 함수 정의
def objective(trial):
    # 하이퍼파라미터 설정
    param = {
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "learning_rate": trial.suggest_float("learning_rate", 0.01, 0.3),
        "n_estimators": trial.suggest_int("n_estimators", 50, 300),
        "objective": "reg:squarederror",
        "use_label_encoder": False,
        "eval_metric": "logloss",
        "early_stopping_rounds": 20
    }

    with mlflow.start_run(nested=True):
            # ✅ 파라미터 기록
            mlflow.log_params(param)

            # XGBoost 모델 학습
            model = XGBRegressor(**param)
            model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)

            # 예측
            y_pred = model.predict_proba(X_test)[:,1]

            # Spark DataFrame으로 Precision@K 계산
            test_pdf = test_df.select("userIndex", "movieIndex", "rating").toPandas()
            test_pdf["y_pred_prob"] = y_pred
            test_pdf.rename(columns={"rating": "y_true"}, inplace=True)

            pred = spark.createDataFrame(test_pdf)
            window_spec = Window.partitionBy("userIndex").orderBy(col("y_pred_prob").desc())
            top_k = pred.withColumn("rank", row_number().over(window_spec)).filter(col("rank") <= K)

            top_k = top_k.withColumn("y_true", (col("y_true") >= 4).cast("int"))
            precision_df = top_k.groupBy("userIndex").agg({"y_true": "avg"})
            avg_precision_at_k = precision_df.selectExpr("avg(`avg(y_true)`) as precision_at_k").collect()[0]["precision_at_k"]

            # ✅ 메트릭 기록
            mlflow.log_metric("precision_at_k", avg_precision_at_k)

            # ✅ 모델 저장
            mlflow.sklearn.log_model(model, "model")

            return -avg_precision_at_k  # Optuna는 최소화 수행하므로 음수로 반환

In [0]:
best_model = XGBClassifier(**study.best_params)
best_model.fit(X_train, y_train, eval_set=[(X_valid, y_valid)], verbose=False)

with mlflow.start_run(run_name="final_best_model"):
    mlflow.log_params(study.best_params)
    mlflow.sklearn.log_model(best_model, "final_model")

# 5. 한계

## 5-1. 안 본 영화 중에서 추천(rating 있다면 본 영화로 판단)

In [0]:
from pyspark.sql.functions import collect_set

# 1. 사용자별 이미 본 영화(movieIndex) 집합 생성 (train + valid + test 혹은 전체 학습 데이터에서)
# 보통은 train_df, valid_df에 있는 user-movie pairs가 '본 영화'임
seen_movies_df = train_df.select("userIndex", "movieIndex") \
    .union(valid_df.select("userIndex", "movieIndex")) \
    .distinct() \
    .groupBy("userIndex") \
    .agg(collect_set("movieIndex").alias("seen_movies"))

# 2. test_pred와 join해서 이미 본 영화 제외
test_pred_filtered = test_pred.join(seen_movies_df, on="userIndex", how="left_outer")

# 3. 이미 본 영화 제외 조건 추가 (seen_movies가 null일 수도 있으니 null-safe 처리)
from pyspark.sql.functions import array_contains

test_pred_filtered = test_pred_filtered.filter(
    (col("seen_movies").isNull()) | (~array_contains(col("seen_movies"), col("movieIndex")))
)

# 4. 사용자별 추천 상위 K개 추출
window_spec = Window.partitionBy("userIndex").orderBy(col("y_pred_prob").desc())

ranked = test_pred_filtered.withColumn("rank", row_number().over(window_spec)) \
                           .filter(col("rank") <= K)

ranked = ranked.withColumn("recommendation", struct(col("movieIndex").alias("movieId"), col("y_pred_prob").alias("pred_rating")))

recommendations = ranked.groupBy("userIndex") \
                        .agg(collect_list("recommendation").alias("recommendations")) \
                        .orderBy("userIndex")

display(recommendations)

## 5-2. 추가 User가 들어왔을때 어떤 영화를 추천할지

1. 신규유저에 영화정보를 바탕으로 기존유저와 매칭

2. 새로운 user_index 추천하지 않음(새로 학습해야함) - 현재 모델에서 user_index가 중요한 역할/ 처음 보는 User_index보고 엉뚱한 예측할 확률 높음

1. Cold Start 유저 처리
신규 유저:

A안: userIndex 제거한 일반 모델로 추천 제공 (장르, 평균 평점 기반)

B안: 기존 유저와 유사도 기반으로 매칭 → 그 유저의 추천을 가져옴 (collaborative filtering 유사)

2. 데이터 업데이트 및 재학습
신규 rating이 수집되면 데이터에 추가

일정 주기 (일간/주간 등)에 따라 모델 재학습

재학습된 모델은 최신화된 추천 반영 가능



1. 시청 영화 바탕으로 유사한 기존 유저 indexr를 찾고,
2. 그 indexr와 시청 영화 바탕으로 추천 영화 예측

In [0]:
# Step 1. 신규 유저가 본 영화 리스트
new_user_seen_movies = [1, 10, 50, 300]
new_user_set = set(new_user_seen_movies)

# Step 2. 기존 유저별 시청 영화 집합 생성
user_movie_sets = ratings.groupBy("userId").agg(collect_set("movieId").alias("movie_set"))

# Step 3. Jaccard 유사도 계산
def jaccard_similarity(set1, set2):
    set1, set2 = set(set1), set(set2)
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return float(intersection) / union if union != 0 else 0.0

jaccard_udf = udf(lambda x: jaccard_similarity(new_user_set, x), DoubleType())

similar_users = user_movie_sets.withColumn("jaccard_sim", jaccard_udf(col("movie_set"))) \
                               .orderBy(desc("jaccard_sim")) \
                               .limit(1)

# Step 4. 유사한 기존 유저의 userId 및 userIndex 추출
top_user_id = similar_users.select("userId").first()["userId"]
top_user_index = user_indexer_model.transform(
    spark.createDataFrame([(top_user_id,)], ["userId"])
).select("userIndex").first()["userIndex"]

In [0]:
# Step 5. 전체 영화 목록에서, 신규 유저가 보지 않은 영화만 필터링
unseen_movies = df_cleaned.filter(~col("movieId").isin(new_user_seen_movies))

# Step 6. 추천 후보 영화들에 대해 movieIndex, features 등 변환 수행
unseen_with_features = pipeline_model.transform(unseen_movies)

# Step 7. 해당 유사 유저 index를 모든 row에 추가
unseen_with_features = unseen_with_features.withColumn("userIndex", lit(top_user_index))

unseen_unique = unseen_with_features.dropDuplicates(["movieId"])

# Step 8. features 컬럼을 numpy 배열로 변환
pdf_unseen = unseen_unique.select("features", "movieId", "title").toPandas()
X_unseen = np.vstack(pdf_unseen["features"].apply(lambda x: x.toArray()))

# Step 9. 모델 예측 (regressor라 predict_proba 아님, predict 사용)
y_pred = model.predict_proba(X_unseen)[:,1]

# Step 10. 예측 결과를 DataFrame에 추가
pdf_unseen["prediction"] = y_pred

# Step 11. 상위 K개 추천
top_k = pdf_unseen.sort_values(by="prediction", ascending=False).head(10)

# 출력
print(top_k[["movieId", "title", "prediction"]])